In [1]:
import cobra
import libsbml
from cobra import Model, Reaction, Metabolite, manipulation
import pandas as pd
import csv
import copy

In [2]:
ios2164_orig = cobra.io.read_sbml_model("../model/ios2164_orig.xml")

#Fix names
cobra.manipulation.modify.escape_ID(ios2164_orig)


Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-03


In [3]:
#Let's insert NGAM for each  of the commpartments based from B&B's Measurements from A. thaliana

In [4]:
#let's add a reaction specific for phloem, which includes only amino acids, starch and sucrose
#Values are from Poolman et al. (2014) and is rescaled from mol/gcw * time to mmol/gcw * time (similar to ios2164 and Arnold's Atmodel)

#Read from csv
read_phl = pd.read_csv('../misc/phloem_sap_comp.csv')
phl_dict = dict(read_phl.values)


model_rxn_list = list()
for items in phl_dict:
    #Update reaction signs first so it's all negative 
    phl_dict[items] = phl_dict[items] * -1
    
    #Retrieve pointer from original model to retrieve metabolite
    model_rxn = ios2164_orig.metabolites.get_by_id(items)
    model_rxn_list.append(model_rxn)

#Update rxns to pointers to orig model
for rxns in model_rxn_list:
    phl_dict[rxns] = phl_dict.pop(rxns.id)

#Generate new phloem reaction
phl_rxn = Reaction('DM_Phloem')
phl_rxn.name = 'export reaction for phloem sap'
phl_rxn.add_metabolites(phl_dict)

ios2164_orig.add_reaction(phl_rxn)
    

Ignoring reaction 'DM_Phloem' since it already exists.


In [5]:
#Add NGAM reactions based on B&B's measurements
#This is a  good way to account for metabolic costs. However, the only limitation would this be would be that the reaction bounds are constant and non-variable

ngam_atp_c = Reaction('ngam_atp_c')
ngam_atp_s = Reaction('ngam_atp_s')
ngam_atp_x = Reaction('ngam_atp_x')
ngam_atp_m = Reaction('ngam_atp_m')

#Values from B&B(2019) are the ff (Units are in umol atp m-2 s-1)
# cytosol	0.0427
# chloroplast	0.1527
# mitochondria	0.0091
# peroxisome	0.0076

ngam_atp_c.name = 'ATP NGAM for Cytosol'
ngam_atp_s.name = 'ATP NGAM for Chloroplast (Plastid)'
ngam_atp_x.name = 'ATP NGAM for Peroxisome'
ngam_atp_m.name = 'ATP NGAM for Mitochondria'

atp_c = ios2164_orig.metabolites.atp_c
adp_c = ios2164_orig.metabolites.adp_c
pi_c = ios2164_orig.metabolites.pi_c

atp_s = ios2164_orig.metabolites.atp_s
adp_s = ios2164_orig.metabolites.adp_s
pi_s = ios2164_orig.metabolites.pi_s

atp_x = ios2164_orig.metabolites.atp_x
adp_x = ios2164_orig.metabolites.adp_x
pi_x = ios2164_orig.metabolites.pi_x

atp_m = ios2164_orig.metabolites.atp_m
adp_m = ios2164_orig.metabolites.adp_m
pi_m = ios2164_orig.metabolites.pi_m

ngam_atp_c.add_metabolites({atp_c:-1, adp_c:1, pi_c:1})
ngam_atp_s.add_metabolites({atp_s:-1, adp_s:1, pi_s:1})
ngam_atp_x.add_metabolites({atp_x:-1, adp_x:1, pi_x:1})
ngam_atp_m.add_metabolites({atp_m:-1, adp_m:1, pi_m:1})



ngam_atp_c.bounds = (0.0427,0.0427)
ngam_atp_s.bounds = (0.1527, 0.1527)
ngam_atp_x.bounds = (0.0076,0.0076)
ngam_atp_m.bounds = (0.0091,0.0091)

ios2164_orig.add_reaction(ngam_atp_c)
ios2164_orig.add_reaction(ngam_atp_s)
ios2164_orig.add_reaction(ngam_atp_x)
ios2164_orig.add_reaction(ngam_atp_m)



Ignoring reaction 'ngam_atp_c' since it already exists.
Ignoring reaction 'ngam_atp_s' since it already exists.
Ignoring reaction 'ngam_atp_x' since it already exists.
Ignoring reaction 'ngam_atp_m' since it already exists.


In [6]:
ngam_atp_c
#It works with a constant NGAM that is approximately 0.0427 umol atp m-2 s-1

Reaction identifier,ngam_atp_c
Name,ATP NGAM for Cytosol
Memory address,0x07f2ca54a3880
Stoichiometry,atp_c --> adp_c + pi_c ATP --> ADP + Phosphate
GPR,
Lower bound,0.0427
Upper bound,0.0427


In [7]:
#Restrict all reactions to white light only
for lights in ios2164_orig.reactions:
    if 'PRISM' in lights.id:
        ios2164_orig.reactions.get_by_id(lights.id).bounds = (-1e9,1e9)
        #Constrain all non-white light reactions to 0
        if 'white' not in lights.id:
            ios2164_orig.reactions.get_by_id(lights.id).bounds = (0,0)


In [8]:
#Add Rubisco Oxygenase-specific reaction to model

#First retrieve the carboxylase reaction
rbc_rxn = ios2164_orig.reactions.RBPCs


#Define reaction. Id is RBPOs
rbo_rxn = Reaction('RBPOs')
rbo_rxn.name = 'Ribulose-biphosphate Oxygenase'

#Components of oxygenase reaction are as follows
#o2[s] + h2o[s] + rb15bp[s] -> 3pg[s] + 2pglyc[s]
#Interestingly, Arnold's model accounts for the generation of protons by the rubisco reaction.
#There's also support from literature on how protonation may also enhance CO2 capture by a plant Cell.
#To accomodate this I'll just add the reaction specifically for both RBO and RBC reactions.

#Retrieve metabolites
o2_s = ios2164_orig.metabolites.o2_s
h2o_s = ios2164_orig.metabolites.h2o_s
rb15bp_s = ios2164_orig.metabolites.rb15bp_s
h_s = ios2164_orig.metabolites.h_s
_3pg_s = ios2164_orig.metabolites.get_by_id('3pg_s')
_2pglyc_s = ios2164_orig.metabolites.get_by_id('2pglyc_s')

#Define Stoichiometry
rbo_rxn.add_metabolites({o2_s:-1.0, h2o_s:-1.0, rb15bp_s:-1, h_s:1, _3pg_s: 1, _2pglyc_s: 1})

#Copy other details from rbc_rxn
rbo_rxn.subsystem = rbc_rxn.subsystem
rbo_rxn.bounds = rbc_rxn.bounds
#Define encoded genes; copied from ios2164 supplementary
rbo_rxn.gene_reaction_rule = rbc_rxn.gene_name_reaction_rule

#Add to model
ios2164_orig.add_reaction(rbo_rxn)

ios2164_orig.repair

ios2164_orig.reactions.RBPOs
        
#addtl refs:
#Long, B. M., Förster, B., Pulsford, S. B., Price, G. D., & Badger, M. R. (2021). 
#Rubisco proton production can drive the elevation of CO2 within condensates and carboxysomes. 
#Proceedings of the National Academy of Sciences, 118(18).

Ignoring reaction 'RBPOs' since it already exists.


Reaction identifier,RBPOs
Name,Ribulose-biphosphate Oxygenase
Memory address,0x07f2ca2a4f490
Stoichiometry,"h2o_s + o2_s + rb15bp_s --> 2pglyc_s + 3pg_s + h_s H2O + O2 + D-Ribulose 1,5-bisphosphate --> 2-Phosphoglycolate + 3-Phospho-D-glycerate + H+"
GPR,LOC_Os01g58020 and LOC_Os05g35330 and LOC_Os11g32770 and LOC_Os12g10580 and LOC_Os10g21280 and...
Lower bound,0.0
Upper bound,1000.0


In [9]:
#Add proton production to original carboxylase reaction
#From B&B (2019), Long et al. (2021)
rbc_rxn.add_metabolites({h_s:1.0})

In [10]:

#Fix original one-cell model
cobra.io.write_sbml_model(ios2164_orig, filename= "../model/ios2164_orig.xml")

In [11]:
#Let's add a specific demand reaction for oxygen

dm_o2 = Reaction('DM_o2(c)')

dm_o2.name = "Demand reaction for oxygen"

o2_c = ios2164_orig.metabolites.o2_c


dm_o2.add_metabolites({o2_c:-1.0})

dm_o2.bounds = (0, 1000)

ios2164_orig.add_reaction(dm_o2)

ios2164_orig.repair()

    

In [12]:
ios2164_m = copy.deepcopy(ios2164_orig)
for met in ios2164_m.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '0'
        met.compartment = str(met.compartment) + '0'

    
        
for rxn in ios2164_m.reactions:
    if "EX_" not in rxn.id: #Exclude exchange reactions
        rxn.id = str(rxn.id)  + '_M'
#     if "EX_" in rxn.id: #Rename media reactions
#         rxn.name = "M " + str(rxn.name)

ios2164_m.repair()


Read LP format model from file /tmp/tmph1rqfw8p.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros


In [13]:
ios2164_bs = copy.deepcopy(ios2164_orig)

for met in ios2164_bs.metabolites:
    if met.compartment != "e": #Extracellular, defines boundary reactions
        met.id = str(met.id) + '1'
        met.compartment = str(met.compartment) + '1'
    
for rxn in ios2164_bs.reactions:
    if "EX_" not in rxn.id: #Exclude exchange reactions
        rxn.id = str(rxn.id)  + '_BS'
#     if "EX_" in rxn.id: #Rename media reactions
#         rxn.name = "BS " + str(rxn.name)


ios2164_bs.repair()

Read LP format model from file /tmp/tmp60lzvvhy.lp
Reading time = 0.01 seconds
: 1999 rows, 4904 columns, 21054 nonzeros


In [14]:
#Merging the BS and M Cells
ios2164_2cell = ios2164_m.merge(ios2164_bs, objective = "sum", inplace = 1)



Ignoring reaction 'EX_co2(e)' since it already exists.
Ignoring reaction 'EX_h2o(e)' since it already exists.
Ignoring reaction 'EX_h(e)' since it already exists.
Ignoring reaction 'EX_no3(e)' since it already exists.
Ignoring reaction 'EX_o2(e)' since it already exists.
Ignoring reaction 'EX_pi(e)' since it already exists.
Ignoring reaction 'EX_sucr(e)' since it already exists.
Ignoring reaction 'EX_fru-B(e)' since it already exists.
Ignoring reaction 'EX_glc-A(e)' since it already exists.
Ignoring reaction 'EX_so4(e)' since it already exists.
Ignoring reaction 'EX_hco3(e)' since it already exists.
Ignoring reaction 'EX_so3(e)' since it already exists.
Ignoring reaction 'EX_h2s(e)' since it already exists.
Ignoring reaction 'EX_nh4(e)' since it already exists.
Ignoring reaction 'EX_asn-L(e)' since it already exists.
Ignoring reaction 'EX_gln-L(e)' since it already exists.
Ignoring reaction 'EX_etoh(e)' since it already exists.
Ignoring reaction 'EX_ac(e)' since it already exists.
Igno

In [15]:
ios2164_2cell

Name,iOS2164
Memory address,0x07f2ca54b34f0
Number of metabolites,3956
Number of reactions,4861
Number of groups,96
Objective expression,2.0*Straw_Biomass - 2.0*Straw_Biomass_reverse_5daed
Compartments,"c0, s0, r0, m0, Extracellular, u0, x0, v0, c1, s1, r1, m1, u1, x1, v1"


In [16]:
#Apparently compartments don't update after  merging so I've updated it na lang after that

ios2164_2cell.compartments = {
 'c0': 'M Cell Cytoplasm',
 's0': 'M Cell Plastid',
 'r0': 'M Cell Endoplasmic_reticulum',
 'm0': 'M Cell Mitochondrion',
 'e': 'Extracellular',
 'u0': 'M Cell Thylakoid',
 'x0': 'M Cell Peroxisome',
 'v0': 'M Cell Vacuole',
 'c1': 'BS Cell Cytoplasm',
 's1': 'BS Cell Plastid',
 'r1': 'BS Cell Endoplasmic_reticulum',
 'm1': 'BS Cell Mitochondrion',
 'u1': 'BS Cell Thylakoid',
 'x1': 'BS Cell Peroxisome',
 'v1': 'BS Cell Vacuole'}

#Also need to rebuild the model medium
ios2164_2cell.repair()


From Blatke & Brautigam (2019):

"The mesophyll and bundle sheath networks are connected by a range of cytosolic transport metabolites including amino acids, sugars (glucose, fructose, sucrose, trehalose, ribose), single phosphorylated sugar (glucose-6-phosphate, glucose-1-phosphate, fructose-6-phosphate, sucrose-6-phosphate), mono-/di-/tri-carboxylic acids (phosphoenolpyruvate, pyruvate, citrate, cis-aconitate, isocitrate, α-ketoglutarate, succinate, fumarate, malate), glyceric acids (2-Phosphoglycerate, 3-Phosphoglycerate), glycolate, glycerate, glyceraldehyde-3-phosphate, di-hydroxyacetone-phosphate and CO2. 

Nucleotides, NAD/NADH, NADP/NADPH, pyrophosphate, inorganic phosphate are not considered as transport metabolites. 

Oxaloacetate has been excluded as transport metabolite since concentrations of oxaloacetate are very low in vivo and it is reasonably unstable in aqueous solutions. 

Other small molecules that can be imported by the bundle sheath from the environment, as well as protons and HCO3-, are not exchanged between the two cell types.

#So bale yung mga organic na soluble lang with reasonably long cellular half-lives.

In [17]:
#Open the initial curated list to get the metabolite ids
transport_rxns = pd.read_csv('../model/initial_list_transport_rxns.csv')
#Ok it gets the reactions now

for item in ios2164_orig.metabolites:
    if item.compartment == "c":
        if item.id in transport_rxns.id.values:
            pd_rxn = Reaction()
            #Generate reaction name
            pd_rxn.id = str(item.id[:-2] + '_pd') #Replaces suffix with '_pd'
            pd_rxn.name = str('Plasmodesmatal transport of ' + item.name)
            pd_rxn.subsystem = str('PD Transport Reaction')
            #Reaction is reversible
            pd_rxn.lower_bound = -1000
            pd_rxn.upper_bound = 1000
            
            
            #Generate metabolite names
            
            
            met_mcell = str(item.id) + '0'
            met_bcell = str(item.id) + '1'
            
            met_mcell = ios2164_2cell.metabolites.get_by_id(met_mcell)
            met_bcell = ios2164_2cell.metabolites.get_by_id(met_bcell)
            pd_rxn.add_metabolites({met_mcell:-1.0, met_bcell:1.0})
            ios2164_2cell.add_reaction(pd_rxn)
            
ios2164_2cell.repair()


#Print how many reactions added to model
printout = 'added ' + str(len(transport_rxns)) + ' PD reactions to model'
print(printout)


added 73 PD reactions to model


In [18]:
#Set all fluxes to 0 muna to make it all easy.
for rxns in ios2164_2cell.exchanges:
    rxn_id = rxns.id
    ios2164_2cell.reactions.get_by_id(rxn_id).bounds = (0, 0)


In [19]:

#Set Sucrose PD to be non-reversible (M -> BS) so as to properly model bulk flow
ios2164_2cell.reactions.get_by_id('sucr_pd').bounds = (0, 1000)



In [20]:
#Modify GPRs to fit GPRs from RNAseq experiment
#Or should I just add specific reactions to accomodate this?
#I'll just add it once I'm writing the scripts for the transgenic ones, so as to streamline the creation of new constraints.

In [21]:
# Change objective function so as to reflect the two cell state


ios2164_2cell.objective = {ios2164_2cell.reactions.Straw_Biomass_M: 1,
                  ios2164_2cell.reactions.Straw_Biomass_BS: 1}
ios2164_2cell.optimize()

print(ios2164_2cell.objective)


Maximize
1.0*Straw_Biomass + 1.0*Straw_Biomass_BS - 1.0*Straw_Biomass_BS_reverse_9ef17 - 1.0*Straw_Biomass_reverse_5daed


/home/rtsantos3/.local/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [22]:
sample_fluxes = ios2164_2cell.optimize()

sample_fluxes

#Should yield infeasible now -- no flux and no  way to generate new ATP to maintain NGAM//

<Solution infeasible at 0x7f2c7e6ec790>

In [23]:
mets = open('../misc/metabolites.csv', 'w')
rxns = open('../misc/reactions.csv', 'w')

met_write = csv.writer(mets)
rxns_write = csv.writer(rxns)

for item in ios2164_2cell.metabolites:
    row = [item.id, item.name]
    met_write.writerow(row)
    

for item in ios2164_2cell.reactions:
    row = [item.id, item.name, item.build_reaction_string(), item.bounds]
    rxns_write.writerow(row)

In [24]:
cobra.io.write_sbml_model(ios2164_2cell, filename= "../model/ios2164_2cell.xml")
#Save Json for mapping
cobra.io.save_json_model(ios2164_2cell, filename="../model/ios2164_2cell.json")
#Save one-cell model to incorporate updates
cobra.io.write_sbml_model(ios2164_orig, filename="../model/ios2164_1cell.xml")